# Read Data

In [11]:
import pandas as pd

def read_csv(file_path):
    """
    Reads a CSV file into a pandas DataFrame.

    Parameters:
    - file_path: str, the path to the CSV file.

    Returns:
    - DataFrame containing the data from the CSV file.
    """
    try:
        data = pd.read_csv(file_path)
        print("CSV file loaded successfully.")
        return data
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
        return None

# Get user input

In [12]:
def get_user_inputs():
    """
    Prompts the user for necessary inputs for 3D projection.

    Returns:
    - x_origin: float, the X coordinate of the cylindrical system's origin.
    - y_origin: float, the Y coordinate of the cylindrical system's origin.
    - axis_of_rotation: str, the chosen axis of rotation ('X' or 'Y').
    - interval: int, the interval (in degrees) for the circular projection.
    """
    while True:
        try:
            x_origin = float(input("Enter the X coordinate of the cylindrical system's origin: "))
            y_origin = float(input("Enter the Y coordinate of the cylindrical system's origin: "))
            axis_of_rotation = input("Specify the axis of rotation (X or Y): ").strip().upper()
            interval = int(input("Enter the interval (in degrees) for circular projection: "))
            
            if axis_of_rotation not in ['X', 'Y']:
                raise ValueError("Axis of rotation must be 'X' or 'Y'.")
            
            return x_origin, y_origin, axis_of_rotation, interval
        except ValueError as e:
            print(f"Invalid input: {e}. Please try again.")


# Project 2D to 3D

In [13]:
import numpy as np

def project_to_3d(data, axis_of_rotation, interval):
    """
    Projects 2D data into 3D space in a cylindrical pattern around the specified axis.

    Parameters:
    - data: DataFrame, containing the 2D data with columns ['ID', 'X', 'Y', 'Stress'].
    - axis_of_rotation: str, the axis ('X' or 'Y') around which to project the data.
    - interval: int, the interval in degrees at which to replicate points around the axis.

    Returns:
    - DataFrame containing the 3D coordinates of the projected points and their Stress values.
    """
    projected_data = []
    angles = np.arange(0, 360, interval)  # Angles in degrees for the circular projection

    for _, row in data.iterrows():
        id, x, y, stress = row['Node Number'], row['X Location (mm)'], row['Y Location (mm)'], row['Equivalent (von-Mises) Stress (MPa)']
        radius = y if axis_of_rotation == 'X' else x

        for angle in angles:
            theta = np.radians(angle)  # Convert angle to radians

            # Calculate new coordinates based on the axis of rotation
            if axis_of_rotation == 'X':
                new_x = x  # X remains constant
                new_y = radius * np.cos(theta)  # Calculate new Y using radius and angle
                new_z = radius * np.sin(theta)  # Calculate new Z using radius and angle
            else:  # axis_of_rotation == 'Y'
                new_x = radius * np.cos(theta)  # Calculate new X using radius and angle
                new_y = y  # Y remains constant
                new_z = radius * np.sin(theta)  # Calculate new Z using radius and angle

            projected_data.append([new_x, new_y, new_z, stress])

    # Create a DataFrame for the 3D projected data
    columns = ['X Location (mm)', 'Y Location (mm)', 'Z Location (mm)', 'Equivalent (von-Mises) Stress (MPa)']
    projected_df = pd.DataFrame(projected_data, columns=columns)

    return projected_df


# Export generated 3D data as CSV

In [14]:
def output_to_csv(projected_data, output_file_path):
    """
    Outputs the projected 3D data to a CSV file.

    Parameters:
    - projected_data: DataFrame, the DataFrame containing the 3D projected data.
    - output_file_path: str, the path where the output CSV file will be saved.
    """
    try:
        projected_data.to_csv(output_file_path, index=False)
        print(f"Projected data successfully saved to '{output_file_path}'.")
    except Exception as e:
        print(f"Failed to save projected data to CSV: {e}")


# Main function

In [15]:
def main(file_path, output_file_path):
    # Load the 2D data from the CSV file
    data = read_csv(file_path)

    # Get user inputs for the origin, axis of rotation, and interval
    x_origin, y_origin, axis_of_rotation, interval = get_user_inputs()

    # Project the 2D data into 3D space
    projected_data = project_to_3d(data, axis_of_rotation, interval)

    # Output the projected data to a new CSV file
    output_to_csv(projected_data, output_file_path)

if __name__ == "__main__":
    file_path = "input_SEQV_2D.csv"  # Specify the path to your CSV file here
    output_file_path = "my_output_csv_file.csv"  # Specify the path for the output CSV file
    main(file_path, output_file_path)

CSV file loaded successfully.
Projected data successfully saved to 'my_output_csv_file.csv'.
